# Aprendizagem por regras

$\quad$ Basicamente a partir de uma base de dados o algoritmo irá gerar regras para classificação dos dados. Inicialmente usaremos o algoritmo _OneR_, o qual irá criar apenas uma regra.

## 1. Algoritmo _OneR_

$\quad$ Nesse algoritmo apenas um atributo dos dados para criar a regra, de preferência, que seja o atributo com menor erro.